In [1]:
def load_imdb_data():
    """
    Charge le dataset IMDb et adapte les colonnes
    pour compatibilité avec l'application
    """
    
    imdb_path = DATA_DIR / 'imdb_complet_avec_tags'
    
    if not imdb_path.exists():
        st.error(f"❌ Fichier IMDb non trouvé : {imdb_path}")
        st.info(f"Chemin attendu : {imdb_path}")
        return None
    
    try:
        # Charger le dataset
        df = pd.read_parquet(imdb_path)
        
        # ==========================================
        # ADAPTATIONS DES COLONNES
        # ==========================================
        
        # 1. Créer un ID numérique si absent
        if 'id' not in df.columns:
            df['id'] = range(1, len(df) + 1)
        
        # 2. Renommer les colonnes pour compatibilité
        column_mapping = {
            'primaryTitle': 'titre',
            'averageRating': 'note',
            'runtimeMinutes': 'durée',
            'numVotes': 'votes'
        }
        
        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df[new_col] = df[old_col]
        
        # 3. Convertir genres (string → liste)
        if 'genres' in df.columns:
            df['genre'] = df['genres'].fillna('').apply(
                lambda x: [g.strip() for g in x.split(',')] if isinstance(x, str) and x else []
            )
        else:
            df['genre'] = [[] for _ in range(len(df))]
        
        # 4. Créer release_date depuis startYear
        if 'startYear' in df.columns:
            df['release_date'] = pd.to_datetime(
                df['startYear'].astype(str) + '-01-01',
                errors='coerce'
            )
        else:
            df['release_date'] = pd.NaT
        
        # 5. Gérer director.name depuis realisateurs
        if 'realisateurs' in df.columns:
            df['director.name'] = df['realisateurs'].apply(format_director)
        else:
            df['director.name'] = 'Inconnu'
        
        # 6. Ajouter synopsis (placeholder ou depuis TMDb si disponible)
        if 'synopsis' not in df.columns:
            if 'tmdb_overview_fr' in df.columns:
                df['synopsis'] = df['tmdb_overview_fr'].fillna('Synopsis non disponible.')
            else:
                df['synopsis'] = 'Synopsis non disponible. Consultez IMDb pour plus d\'informations.'
        
        # 7. Gérer tmdb_id
        if 'tmdb_id' not in df.columns:
            # Créer un ID factice depuis tconst
            df['tmdb_id'] = df['tconst'].apply(lambda x: abs(hash(x)) % 1000000)
        
        # 8. Calculer a_laffiche (films récents)
        if 'startYear' in df.columns:
            current_year = datetime.now().year
            df['a_laffiche'] = df['startYear'] >= (current_year - 2)  # Films des 2 dernières années
        else:
            df['a_laffiche'] = False
        
        # 9. Nettoyer les valeurs manquantes
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)
        df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce').fillna(90)
        df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0)
        
        # 10. Filtrer les films de qualité minimale
        df = df[
            (df['rating'] > 0) &
            (df['votes'] >= 100) &
            (df['runtime'] >= 60)
        ].copy()
        
        # ==========================================
        # COLONNES FINALES
        # ==========================================
        colonnes_base = [
            'id', 'tconst', 'title', 'release_date', 'genre', 
            'runtime', 'rating', 'votes', 'director.name', 
            'synopsis', 'tmdb_id', 'a_laffiche'
        ]
        
        # Ajouter tags et recommandations si elles existent
        if 'tags' in df.columns:
            colonnes_base.append('tags')
        if 'recommandations' in df.columns:
            colonnes_base.append('recommandations')
        
        # Garder seulement les colonnes qui existent
        colonnes_disponibles = [col for col in colonnes_base if col in df.columns]
        df = df[colonnes_disponibles].copy()
        
        # Réinitialiser l'index
        df = df.reset_index(drop=True)
        
        return df
        
    except Exception as e:
        st.error(f"❌ Erreur lors du chargement IMDb : {e}")
        st.info(f"Type d'erreur : {type(e).__name__}")
        import traceback
        st.code(traceback.format_exc())
        return None

In [4]:
from utils import get_project_root
PROJECT_ROOT = get_project_root()
DATA_DIR = PROJECT_ROOT / "data" / "processed"
df= load_imdb_data()

ModuleNotFoundError: No module named 'utils'